In [ ]:
!pip3 install spacy

In [ ]:
import spacy
from spacy.tokens import DocBin
import random
import string

In [ ]:


def normalize_text(text):
    return text.strip().lower().translate(str.maketrans('', '', string.punctuation))

def create_training_example(text, entity_text, label, placetype, placelabel):
    entities = []
    if entity_text:
        start = text.find(entity_text)
        if start != -1:
            entities.append((start, start + len(entity_text), label))
    if placetype:
        start = text.find(placetype)
        if start != -1:
            entities.append((start, start + len(placetype), placelabel))
    return (text, {"entities": entities})

not llm 
small model 

text2vec + (cnn + encoder)

test_sentences = [
    # Known patterns
    
    "Any restaurant near piassa?",
    "Looking for a parking spot in bole medhane alem",
    "Is there an Ethiopian restaurant around shiro meda?",
    "Can you find me a cafe close to stadium?",
    "Show me a Chinese restaurant in aware",
    "Where’s a fast food place near kasanchis?",
    "Need a vegan restaurant around tafo",
    "Is there a parking spot within atlas?",
    "Find restaurant in bisrate gabriel",
    "Where to eat Italian food near meskel flower?",

    # New modifiers and phrasing
    "How do I get to a cafe not far from yeka abado meskelegna?",
    "Suggest a place to park near ayat condominum",
    "Are there any restaurants next to bella?",
    "I'm craving fast food near lancha",
    "Know any good vegan spots around raguael?",
    "A place to grab coffee near bole bulbula?",
    "Where can I chill in a cafe near hayararat?",
    "I need somewhere to eat in gerji mebrat hayl",
    "Cheap restaurant options around golla please",
    "Could you recommend a parking area in gofa sefer?",

    # Trickier phrasings
    "I'm looking for some Ethiopian food by qorie",
    "Show me a parking garage close to tatek industry zone",
    "Any good breakfast cafes in saris abo?",
    "Is a parking spot available near geter menafesha kotebe?",
    "Best Chinese near dejach wube sefer?",
]

Action = [
    "Is there any" , "can you find me" , "show me" , "need a","Where is the",
          "Is there a","Find me", "I need a", "Find", "Search for", "Where can I get", "I need", "Looking for" , "Where can I find"]
Place_type = ["Shoping mall","airport","art gallery","bakery","bicycle store","book store","cafe","campground park","clothing store","department store","drugstore","electronic store","electronics store","food establishment","food store","furniture store","general contractor","hair care","hardware store","home goods store","hospital","jewelry store","laundry","liquor store","local government office","lodging","lodging establishment", "loding establishment","movie theater","night club","office","painter","park","parking establishment","pharmacy","plumber","secondary school","shopping mall","storage store","store","supermarket","tourist attraction","travel agency", "university","veterinary care","Alcoholic beverage wholesaler","Apartment Building","Auto Spare parts","Beverage distributor","Bolo","Building","Building design company","Building materials store","Bus depot","Bus stop","Business Center","Business development service","Camping farm", "Chemical exporter","Chemical manufacturer","Church","Cinema equipment supplier","Clinic","Condominiums","Construction machine dealer","Construction machine rental service","Construction material wholesaler","Corporate entertainment service","Cultural association","Djibuti St","Door manufacturer","Driving School","Economic consultant","Electrical Garage","Establishment","Exporter","Food processing company","Fuel supplier","Gas station","Guest house","Home help service agency","Housing complex", "Information services","Insurance company","International school","Laboratory","Manufacturer","Marble supplier","Media house","Medical equipment supplier","Mesjid","Monument","Petroleum products company","Police Station","Restaurant","School","Sparepart","Statue","Textile mill","Trade centre","Traditional market","Villa","accounting establishment","airport","amusement park","apartment","aquarium establishment","art gallery","atm","bakery","bakery store","bank", "bar","beauty salon","bolo","building","buliding","bus station","cafe","cafe","campground","campground park","car dealer","car decor","car rental","car repair","car repair establishment","car sales","car wash","casino establishment","cemetery","church","clothing store","college","convenience store","courthouse establishment","dentist","drugstore","electrical store","electrician establishment","electronics store","embassy","establishment","finance establishment","finance store","fire station","florist","food establishment","food store","foundation","funeral home","furniture store","garage","gas station","general contractor","grocery","guest house","gym","hair care","hair care store","health","health store","hindu temple","home good stores","home goods","home goods store","home goods store", "hospital","hotel","insurance agency","insurance company","jewelry store","landmark establishment","laundry establishment","lawyer establishment","library","liquor store","local government office","locksmith","lodging","lodging establishment","lodging store","mall","market","mosque","movie rental","movie theater","movie_rental establishment","moving company","museum","night club","office","painter establishment","park","parking","parking establishment","pet store","pharmacy","physiotherapist","place of worship","police establishment","police station","post office","premise establishment","primary school","rail station","real estate agency","restaurant","round about","school","school establishment","secondary school","shoe store","shop","shopping mall","spa","spa health","square","storage","store","supermarket","tailor center","taxi stand","tourist attraction","tourist attraction establishment","training center","transit station","travel agency","university","university library","veterinary care"]
Location_modifier = ["in", "near", "close to", "around", "within"]
Location = ["legehar","aware","africa union","piassa","meshualekia","yerer","shola","addisu gebeya","sar bet","bole arabsa","beklo bet","ayat condominum","nefas silk","woji","gedam sefer","ayat medhane alem","bisrate gabriel","bole medhane alem","sumale tera","asko","habte giyorgis","bella","iri bekentu","meskel square","qorie","dejach wube sefer","medhane alem","stadium","sidist kilo","gurara","lancha","gebre christos","kasanchis","atlas","senga tera","amist kilo","lam beret","gofa sefer","weyira sefer","atkilt tera","shiro meda","haya arat","bambis", "signal","shogele","saris abo","wello sefer","golla","wesen","biherawi","bole bulbula","goro","gerji mebrat hayl","arada","bole","meskel flower","tatek industry zone", "yeka abado meskelegna", "tafo ccd", "tafo", "geter menafesha kotebe", "somali tera", "raguael"]

# Generate training examples
train_data = []
for action in Action:
    for place in Place_type:
        for mod in Location_modifier:
            for loc in Location:
                text = normalize_text(f"{action} {place} {mod} {loc}")
                train_data.append(create_training_example(text, loc, "LOC", place, "TYPE"))




In [ ]:
nlp = spacy.blank("en")
doc_bin = DocBin()
for text, annot in train_data:
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label)
        if span: ents.append(span)
    doc.ents = ents
    doc_bin.add(doc)
doc_bin.to_disk("train.spacy")